In [1]:
import pandas as pd
import numpy as np

In [2]:
DISPLAY_DF = True
DISPLAY_CALC = True
ROUNDING_POINT = 6

### Fill Problem Information (Manual) and Parse into DataFrames (Automated)

In [3]:
units_raw = {
    'unit'  :['U1'       ],
    'role'  :['standard' ],
    'inlet' :[['S1']     ],
    'outlet':[['S2']     ],
    'ratio' :[{}         ]
}

streams_raw = {
    'stream'    :['S1'           ,'S2'             ],
    'species'   :[['H2','N2']    ,['H2','N2','NH3']],
    'mole_ratio':[{'H2':0.672}   ,{}               ],
    'mole_flow' :[{'H2':100,'total':214}  ,{}               ]
}

reactions_raw = {
    'reaction':["N2+3H2=2NH3"            ],
    'unit'    :['U1'                     ],
    'f'       :[{'H2':0.139}             ],
    'ksee'    :[np.nan                   ],
    'um'      :[{'H2':-3,'N2':-1,'NH3':2}]
}

# Create unit and stream DataFrames
units = pd.DataFrame(units_raw).set_index('unit')
streams = pd.DataFrame(streams_raw).set_index('stream')
reactions = pd.DataFrame(reactions_raw).set_index('reaction')

In [4]:
# Parse upstream and downstream from inlet and outlet
def get_related_unit(S, col):
    #explodes the relevant column (inlet or outlet), filters for the rows there the new column matches S.name, then returns the index of the first match (or None if there were no matches)
    return next(iter(units.explode(col)[units.explode(col)[col] == S.name].index), None)

streams['upstream'] = streams.apply(get_related_unit, args=(['outlet']), axis = 1)
streams['downstream'] = streams.apply(get_related_unit, args=(['inlet']), axis = 1)

if DISPLAY_DF: display(units)
if DISPLAY_DF: display(streams)
if DISPLAY_DF: display(reactions)

#Code to get a DataFrame of the defined unit ratios for spliter units
#pd.json_normalize(units[units.role=="split"].ratio).set_index(units[units.role=="split"].index)

,role,inlet,outlet,ratio
unit,,,,
U1,standard,[S1],[S2],{}


,species,mole_ratio,mole_flow,upstream,downstream
stream,,,,,
S1,"[H2, N2]",{'H2': 0.672},"{'H2': 100, 'total': 214}",None,U1
S2,"[H2, N2, NH3]",{},{},U1,None


,unit,f,ksee,um
reaction,,,,
N2+3H2=2NH3,U1,0.139,NaN,"{'H2': -3, 'N2': -1, 'NH3': 2}"


In [5]:
# get a list of the component species
species = sorted(streams.explode('species').species.unique())

# parse 
n = pd.DataFrame(index=species+['total'])
n.index.name = 'n'
for S in streams.index:
    n[S] = [streams.mole_flow[S].get(s, (np.nan if (s in (streams.species[S]+['total'])) else 0.00)) for s in n.index]

x = pd.DataFrame(index=species)
x.index.name = 'x'
for S in streams.index:
    x[S] = [streams.mole_ratio[S].get(s, np.nan if (s in streams.species[S]) else 0.00) for s in x.index]

if DISPLAY_DF: display(n)
if DISPLAY_DF: display(x)

,S1,S2
n,,
H2,100.0,NaN
N2,NaN,NaN
NH3,0.0,NaN
total,214.0,NaN


,S1,S2
x,,
H2,0.672,NaN
N2,NaN,NaN
NH3,0.000,NaN


### Define Solving functions

In [6]:
def complete_ratios(df):
    changed = False
    for col in df.columns[df.isna().sum() == 1]:
        for i in df.index[df[col].isna()]:
            changed = True
            df.loc[i, col] = np.round(1 - sum([X for X in df.drop(i)[col]]),ROUNDING_POINT)
            if DISPLAY_CALC: print(f"{df.index.name}.{col}.{i} = 1 - {[f"{df.index.name}.{col}.{I}" for I in df.drop(i).index if df.loc[I, col] != 0.00]} = {df.loc[i,col]}")
    return changed
def complete_mole_ratios(): return complete_ratios(x)

def totals_from_components(df_col):
    sans_total = df_col.drop('total')
    if np.isnan(df_col.total) and not sans_total.isna().any():
        result = np.round(sum(sans_total),ROUNDING_POINT) 
        if DISPLAY_CALC: print(f"{df_col.index.name}.{df_col.name} = sum({[f"{df_col.index.name}.{df_col.name}.{I}" for I in sans_total.index if df_col.loc[I] != 0.00]}) = {result}")
        return result
    else:
        return df_col.total
def n_from_all_n_s(): n.loc['total'] = n.apply(totals_from_components, axis=0)

def component_from_total_and_ratio(a_df, r_df):
    for S in a_df.columns[~np.isnan(a_df.loc['total'])]:
        for i in r_df.index[~r_df[S].isna() & a_df.drop('total')[S].isna()]:
            a_df.loc[i, S] = np.round(a_df[S].total * r_df.loc[i, S],ROUNDING_POINT)
            if DISPLAY_CALC: print(f"{a_df.index.name}.{S}.{i} = {a_df.index.name}.{S} * {r_df.index.name}.{S}.{i} = {a_df.loc[i, S]}")
def n_s_from_n_and_x_s(): return component_from_total_and_ratio(n, x)

def ratio_from_component_and_total(a_df, r_df):
    for S in a_df.columns[~np.isnan(a_df.loc['total'])]:
        for i in r_df.index[r_df[S].isna() & ~a_df.drop('total')[S].isna()]:
            r_df.loc[i, S] = np.round(a_df.loc[i, S] / a_df[S].total,ROUNDING_POINT)
            if DISPLAY_CALC: print(f"{r_df.index.name}.{S}.{i} = {a_df.index.name}.{S}.{i} / {r_df.index.name}.{S} = {r_df.loc[i, S]}")
def x_s_from_n_s_and_n(): return ratio_from_component_and_total(n, x)

def total_from_component_and_ratio(a_df, r_df):
    for S in a_df.columns[np.isnan(a_df.loc['total'])]:
        for i in r_df.index[~a_df.drop('total')[S].isna() & r_df[S] > 0]:
            a_df[S].total = np.round(a_df.loc[i, S] / r_df.loc[i, S],ROUNDING_POINT)
            if DISPLAY_CALC: print(f"{a_df.index.name}.{S} = {a_df.index.name}.{S}.{i} / {r_df.index.name}.{S}.{i} = {a_df[S].total}")
            break
def n_from_n_s_and_x_s(): return total_from_component_and_ratio(n, x)

### Set basis, complete ratios, and calculate individual flow rates

In [7]:
n.S1.total = streams.mole_flow.S1['total']
complete_mole_ratios()
n_s_from_n_and_x_s()



display(n)
display(x)

x.S1.N2 = 1 - ['x.S1.H2'] = 0.328
n.S1.N2 = n.S1 * x.S1.N2 = 70.192


,S1,S2
n,,
H2,100.000,NaN
N2,70.192,NaN
NH3,0.000,NaN
total,214.000,NaN


,S1,S2
x,,
H2,0.672,NaN
N2,0.328,NaN
NH3,0.000,NaN
